# Run a Hail notebook in the background

On the *All of US* Workbench, users are logged out after 30 minutes of inactivity. For long running notebooks, such as Hail jobs, this means that users may not have all notebook cells populated even though the notebook continues to run to completion.

<div class="alert alert-block alert-success">
    <b>If you wish to capture all notebook cell outputs</b>, use this notebook to run your long-running notebook.<p>But also note that your analysis <a href="https://support.terra.bio/hc/en-us/articles/360029761352-Preventing-runaway-costs-with-notebook-auto-pause-#h_de5698f5-3c82-4763-aaaf-ea7df6a1869c">the cluster will autopause after 24 hours</a> <b>(right click)</b>. To prevent your cluster from shutting down if your background notebook execution takes longer than 24 hours, be sure to log in and start a notebook, any notebook, within the workspace where the background job is running to reset the autopause timer.</p>
</div>

How to use this notebook:
1. Copy this notebook to the workspace that contains the long-running notebook you wish to run in the background.
1. Edit the filename in `NOTEBOOK_TO_RUN` to be the name of the notebook in this workspace you wish to run.
1. Use menu `Cell -> Run All` to tell the kernel to run this notebook to completion.
1. Close this tab and do other work (note: when you close the tab, the outputs in *this* notebook will no longer be updated, and that is fine, because what we really want are the outputs in the notebook running in the background).
1. When the background notebook execution is complete:
    1. Your Cloud Analysis Environment will pause (if you have no other notebooks open and you did not change the ["Automatically pause after idle"](https://support.terra.bio/hc/en-us/articles/360029761352-Preventing-runaway-costs-with-notebook-auto-pause-#h_de5698f5-3c82-4763-aaaf-ea7df6a1869c) **(right click)** setting).
    1. You will see a copy of your notebook in the notebooks tab with a date and timestamp suffix --> this is the output file.
    1. You will also see the Hail log, if applicable, in `gs://<workspace bucket>/hail-logs/YYYYMMDD/hail*.log`

If you check back, and you do not see the output notebook, the background job is still running.
* To confirm this, first notice whether your Cloud Analysis Environment is still running (green dot next to thunder/cloud icon).
* If it is still running, from the terminal:
  * for Hail, run `yarn application -list` to see that a spark context is still processing data.
  * for Python or R run `top` to see the processes using CPU and memory.

## Setup

In [ ]:
import os
# suppress warning info
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or '3' to suppress both INFO and WARNING logs

In [ ]:
import glob
import gzip
import nbformat
from nbconvert.preprocessors import CellExecutionError
from nbconvert.preprocessors import ExecutePreprocessor
# import os
import shutil
import tensorflow as tf
import time

In [ ]:
# Uncomment these lines if you want to see your list of notebooks in the current working directory.
#!pwd
#!ls

<div class="alert alert-block alert-warning">
<b>Change the cell below</b> to:
    <ol>
        <li>specify the name of the notebook in this workspace that you wish to run in the background</li>
        <li>set the kernel to <b>PYTHON</b> if you have a Hail or Python notebook. Set the kernel to <b>R</b> if you have an R notebook.</li>
    </ol>
There is no need to modify any of the other cells.
</div>

In [ ]:
#---[ Change this to be the name of the notebook in the current working directory that you wish to run. ]-----------
NOTEBOOK_TO_RUN = 'your_notebook.ipynb'

#---[ Change the following to 'PYTHON' if you have a Python notebook, or to 'R' if you have an R notebook.
KERNEL = 'Python'

## Programmatically set output paths

Formulate a new filename for the output notebook so that it include a date and timestamp for when it was executed.

In [ ]:
TIMESTAMP_FILE_SUFFIX = time.strftime('_%Y%m%d_%H%M%S.ipynb')
OUTPUT_NOTEBOOK = NOTEBOOK_TO_RUN.replace('.ipynb', TIMESTAMP_FILE_SUFFIX)

print(f'Executed notebook will be written to filename "{OUTPUT_NOTEBOOK}" on the local disk and the workspace bucket.')

In [ ]:
DATESTAMP = time.strftime('%Y%m%d')
HAIL_LOG_DIR_FOR_PROVENANCE = os.path.join(os.getenv('WORKSPACE_BUCKET'), 'hail-logs', DATESTAMP)

print(f'Hail logs, if any, will be copied to {HAIL_LOG_DIR_FOR_PROVENANCE}')

## Execute the notebook and capture provenance

In [ ]:
def get_kernel(kernel):
    return 'ir' if kernel.lower() == 'r' else 'python3'

KERNEL_NAME = get_kernel(KERNEL)

<div class="alert alert-block alert-info">
The next cell will use <kbd>nbconvert</kbd> to run the notebook in the background until it completes (or yields an error).
    <ul>
    <li>It will capture all notebook outputs as a separate notebook file and store it in the workspace bucket.</li>
    <li>Hail logs, if any exist, will be copied from the execution directory on the local disk to the workspace bucket.</li>
    </ul>
</div>

In [ ]:
# See also https://nbconvert.readthedocs.io/en/latest/execute_api.html
with open(NOTEBOOK_TO_RUN) as f_in:
    nb = nbformat.read(f_in, as_version=4)
    ep = ExecutePreprocessor(timeout=-1, kernel_name=KERNEL_NAME)
    try:
        out = ep.preprocess(nb, {'metadata': {'path': ''}})
    except CellExecutionError:
        out = None
        print(f'''Error executing the notebook "{NOTEBOOK_TO_RUN}".
        See notebook "{OUTPUT_NOTEBOOK}" for the traceback.''')
    finally:
        with open(OUTPUT_NOTEBOOK, mode='w', encoding='utf-8') as f_out:
            nbformat.write(nb, f_out)
        # Save the executed notebook to the workspace bucket.
        output_notebook_path = os.path.join(os.getenv('WORKSPACE_BUCKET'), 'notebooks', OUTPUT_NOTEBOOK)
        tf.io.gfile.copy(src=OUTPUT_NOTEBOOK, dst=output_notebook_path)
        print(f'Wrote executed notebook to {output_notebook_path}')

# Save the hail logs, if any, to the workspace bucket.
for hail_log in glob.glob('hail*.log'):
    with open(hail_log, 'rb') as f_in:
        compressed_hail_log = f'{hail_log}.gz'
        with gzip.open(compressed_hail_log, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    hail_log_path = os.path.join(HAIL_LOG_DIR_FOR_PROVENANCE, compressed_hail_log)
    if not tf.io.gfile.exists(hail_log_path):
        tf.io.gfile.copy(src=compressed_hail_log, dst=hail_log_path)
        print(f'Wrote hail log to {hail_log_path}')